In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, bias=False) #input:28x28x1, output:26x26x8, RF:3,
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(8)
        self.d1 = nn.Dropout(0.03)  

        self.conv2 = nn.Conv2d(8, 8, 3, bias=False) #input:26x26x8, output:24x24x8, RF:5
        self.relu1 = nn.ReLU()
        self.bn2 = nn.BatchNorm2d(8)
        self.d2 = nn.Dropout(0.03) 

        self.conv3 = nn.Conv2d(8, 8, 3, bias=False) #input:24x24x8, output:22x22x8, RF:7
        self.relu1 = nn.ReLU()
        self.bn3 = nn.BatchNorm2d(8)
        self.d3 = nn.Dropout(0.03)  

        self.conv4 = nn.Conv2d(8, 8, 3, bias=False) #input:22x22x8, output:20x20x8, RF:9
        self.relu1 = nn.ReLU()
        self.bn4 = nn.BatchNorm2d(8)
        self.d4 = nn.Dropout(0.03) 

        self.pool1 = nn.MaxPool2d(2, 2) # input:20x20x8 output:10x10x8 RF:10

        self.conv5 = nn.Conv2d(8, 16, 3, bias=False) #input:10x10x8, output:8x8x16, RF:18
        self.relu1 = nn.ReLU()
        self.bn5 = nn.BatchNorm2d(16)
        self.d5 = nn.Dropout(0.03)  
        
        self.conv6 = nn.Conv2d(16, 16, 3, bias=False) #input:8x8x16, output:6x6x16, RF:20
        self.relu1 = nn.ReLU()
        self.bn6 = nn.BatchNorm2d(16)
        self.d6 = nn.Dropout(0.03) #5 % probability 

        self.conv7 = nn.Conv2d(16, 16, 3, bias=False) #input:6x6x16, output:4x4x16, RF:22
        self.relu1 = nn.ReLU()
        self.bn7 = nn.BatchNorm2d(16)
        self.d7 = nn.Dropout(0.03) #5 % probability 

        self.conv8 = nn.Conv2d(16, 16, 3, bias=False) #input:4x4x16, output:2x2x16, RF:24
        self.relu1 = nn.ReLU()
        self.bn8 = nn.BatchNorm2d(16)
        self.d8 = nn.Dropout(0.03) #5 % probability 
        
        self.conv9 = nn.Conv2d(16, 10, 1, bias=False) #input:2x2x16, output:2x2x10, RF:26
        

    def forward(self, x):
        x = F.relu(self.bn1(self.d1(self.conv1(x))))
        x = F.relu(self.bn2(self.d2(self.conv2(x))))
        x = F.relu(self.bn3(self.d3(self.conv3(x))))
        x = self.pool1(F.relu(self.bn4(self.d4(self.conv4(x)))))

        x = F.relu(self.bn5(self.d5(self.conv5(x))))
        x = F.relu(self.bn6(self.d6(self.conv6(x))))
        x = F.relu(self.bn7(self.d7(self.conv7(x))))
        x = self.conv9(F.relu(self.bn8(self.d8(self.conv8(x)))))

        x = F.avg_pool2d(x, 2)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [18]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
           Dropout-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
            Conv2d-4            [-1, 8, 24, 24]             576
           Dropout-5            [-1, 8, 24, 24]               0
       BatchNorm2d-6            [-1, 8, 24, 24]              16
            Conv2d-7            [-1, 8, 22, 22]             576
           Dropout-8            [-1, 8, 22, 22]               0
       BatchNorm2d-9            [-1, 8, 22, 22]              16
           Conv2d-10            [-1, 8, 20, 20]             576
          Dropout-11            [-1, 8, 20, 20]               0
      BatchNorm2d-12            [-1, 8, 20, 20]              16
        MaxPool2d-13            [-1, 8, 10, 10]               0
           Conv2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:62: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:
torch.manual_seed(1)
batch_size = 32

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

In [0]:
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    correct = 0
    train_loss = 0 
    pbar =  tqdm_notebook(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        train_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')

    train_loss /= len(train_loader.dataset)

    print('Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        train_loss, correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [21]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in tnrange(0, 20):
    print("\nEpoch:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


Epoch: 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:62: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train set: Average loss: 0.2072, Accuracy: 56659/60000 (94.43%)
Test set: Average loss: 0.0602, Accuracy: 9802/10000 (98.02%)

Epoch: 1


Train set: Average loss: 0.0681, Accuracy: 58792/60000 (97.99%)
Test set: Average loss: 0.0384, Accuracy: 9886/10000 (98.86%)

Epoch: 2


Train set: Average loss: 0.0579, Accuracy: 58922/60000 (98.20%)
Test set: Average loss: 0.0341, Accuracy: 9886/10000 (98.86%)

Epoch: 3


...

Train set: Average loss: 0.0245, Accuracy: 59537/60000 (99.23%)
Test set: Average loss: 0.0204, Accuracy: 9930/10000 (99.30%)

Epoch: 16


Train set: Average loss: 0.0230, Accuracy: 59585/60000 (99.31%)
Test set: Average loss: 0.0234, Accuracy: 9933/10000 (99.33%)

Epoch: 17


Train set: Average loss: 0.0229, Accuracy: 59555/60000 (99.26%)
Test set: Average loss: 0.0230, Accuracy: 9927/10000 (99.27%)

Epoch: 18


Train set: Average loss: 0.0216, Accuracy: 59579/60000 (99.30%)
Test set: Average loss: 0.0209, Accuracy: 9929/10000 (99.29%)

Epoch: 19


Train set: Average loss: 0.0210, Accuracy: 59602/60000 (99.34%)
Test set: Average loss: 0.0215, Accuracy: 9931/10000 (99.31%)
